# WMT21 Translation

https://huggingface.co/facebook/wmt21-dense-24-wide-x-en

WMT 21 X-En is a 4.7B multilingual encoder-decoder (seq-to-seq) model trained for one-to-many multilingual translation. It was introduced in this paper and first released in this repository.

The model can directly translate text from 7 languages: Hausa (ha), Icelandic (is), Japanese (ja), Czech (cs), Russian (ru), Chinese (zh), German (de) to English.

To translate into a target language, the target language id is forced as the first generated token. To force the target language id as the first generated token, pass the forced_bos_token_id parameter to the generate method.

Since the model was trained with domain tags, you should prepend them to the input as well.

"wmtdata newsdomain": Use for sentences in the news domain
"wmtdata otherdomain": Use for sentences in all other domain

In [1]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#https://huggingface.co/facebook/wmt21-dense-24-wide-en-x
model_name="facebook/wmt21-dense-24-wide-en-x"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def translate(model, tokenizer, textinput):
    newsdomain="wmtdata newsdomain "
    otherdomain="wmtdata otherdomain "
    textinput = newsdomain+textinput
    inputs = tokenizer(textinput, return_tensors="pt")

    # translate English to Chinese
    generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id("zh")) #max_new_tokens
    result=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(result)
    return result

In [3]:
textinput="Israeli troops scour Gaza’s al-Shifa Hospital for evidence of Hamas."
result = translate(model, tokenizer, textinput)

/Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['以色列军队搜查加沙的希法医院,寻找哈马斯存在的证据。']


In [4]:
textinput="The Israeli raid of the Gaza Strip’s largest hospital stretched into its second day Thursday as troops searched for evidence of the extensive Hamas infrastructure that Israeli and U.S. officials have said lies beneath the facility. The Israel Defense Forces said Thursday that searches had uncovered the body of a captive Israeli woman in a house near the hospital, along with weapons. On Wednesday, the IDF released photographs and video of small caches of weapons it said belonged to Hamas. The military added to its case Thursday with a photo and video of a rough cavity that it described as an “operational tunnel shaft.” The Washington Post verified the location of the shaft inside the al-Shifa Hospital complex but could not verify where the opening led or what its purpose might be."
result = translate(model, tokenizer, textinput)

['以色列对加沙地带最大医院的突袭行动于周四进入第二天,以军搜查了哈马斯在该医院地下拥有大量基础设施的证据,以色列和美国官员称这些基础设施位于该设施之下。以色列国防军周四表示,搜查行动在医院附近的一所房屋中发现了一名被俘以色列妇女的尸体,以及武器。周三,以色列国防军发布了据称属于哈马斯的小型武器藏匿处的照片和视频。军方周四在其案例中添加了一张照片和一段视频,该照片和视频显示了一个粗糙的空洞,该空洞被描述为"操作隧道竖井"。《华盛顿邮报》核实了该竖井在al-Shifa医院建筑群内的位置,但无法核实该开口通向何处,也无法核实其目的。']


In [5]:
device='mps'
model.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(128009, 2048, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(128009, 2048, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=2048, out_features=16384, bias=True)
          (fc2): Linear(in_features=16384, out_features=2048, bias=True)
          (final_layer_norm): LayerNo

In [6]:
def translate(model, tokenizer, textinput, device):
    newsdomain="wmtdata newsdomain "
    otherdomain="wmtdata otherdomain "
    textinput = newsdomain+textinput
    inputs = tokenizer(textinput, return_tensors="pt")
    inputs.to(device)

    # translate English to Chinese
    generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id("zh")) #max_new_tokens
    result=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(result)
    return result

In [7]:
textinput="The Israeli raid of the Gaza Strip’s largest hospital stretched into its second day Thursday as troops searched for evidence of the extensive Hamas infrastructure that Israeli and U.S. officials have said lies beneath the facility. The Israel Defense Forces said Thursday that searches had uncovered the body of a captive Israeli woman in a house near the hospital, along with weapons. On Wednesday, the IDF released photographs and video of small caches of weapons it said belonged to Hamas. The military added to its case Thursday with a photo and video of a rough cavity that it described as an “operational tunnel shaft.” The Washington Post verified the location of the shaft inside the al-Shifa Hospital complex but could not verify where the opening led or what its purpose might be."
result = translate(model, tokenizer, textinput, device)

/Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/utils.py:690: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343686130/work/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/beam_search.py:358: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/w

['以色列对加沙地带最大医院的突袭行动于周四进入第二天,以军搜查了哈马斯在该医院地下拥有大量基础设施的证据,以色列和美国官员称这些基础设施位于该设施之下。以色列国防军周四表示,搜查行动在医院附近的一所房屋中发现了一名被俘以色列妇女的尸体,以及武器。周三,以色列国防军发布了据称属于哈马斯的小型武器藏匿处的照片和视频。军方周四在其案例中添加了一张照片和一段视频,该照片和视频显示了一个粗糙的空洞,该空洞被描述为"操作隧道竖井"。《华盛顿邮报》核实了该竖井在al-Shifa医院建筑群内的位置,但无法核实该开口通向何处,也无法核实其目的。']
